## Submitting jobs to stoomboot ##
Joran Angevaare <j.angevaare@nikhef.nl> 

first lets see if I have jobs running

In [1]:
import numpy as np

In [2]:
# !qstat -u jorana | grep jorana | grep Q | wc -l
# !qstat -u jorana | grep jorana | grep R | wc -l

# !qstat | grep jorana && qstat -u jorana


# !qstat

# !qstat -u jorana && echo total: && qstat -u jorana | grep jorana | wc -l && echo running: && qstat -u jorana | grep jorana | grep R |wc -l && echo que: && qstat -u jorana |grep jorana | grep Q | wc -l

In [3]:
# for job in range(4392928, 4393134+1):
#     !qdel $job

In [4]:
priors_from = ['migdal_wide']

In [ ]:
import pandas as pd

In [ ]:
finished = pd.read_csv('../finished_sims.csv')
(np.abs(finished.mw.astype(float) - 0.5) < 0.1) & (finished.config_sigma.astype(int) == -38) & (
    finished.n_fit_parameters == 2) & (finished.config_nlive == 1000)
finished

In [ ]:
np.abs(finished.mw.astype(float) - 0.5) < 0.5

In [ ]:
import uuid
def unique_hash():
    return uuid.uuid4().hex[15:]
unique_hash()

In [ ]:
priors_from = ['migdal_wide']
## PROFILER
prof_dir = '/user/jorana/project/jorana/software/DD_DM_targets/submit_to_stoomboot/profiles'
# tol = 0.8
# note = "TEST"
note = "migdal_04_03"
i,k=0,0
verbose = 1
save_intm = 'yes'
# n_cores = 4
# for cores in range(4,32,4):
for n_rep in np.linspace(0,1,1):
 for save_intm in ["no"]: #, "yes"
  shielding = save_intm  
  for p in priors_from:
   for tol in [0.9]:   
    for shielding in ["yes"]:
     for nlive in [1000]:
      for m in [5,1,0.5, 0.2]:
       if m == 5: sigmas = [-41,-40,-39]
       if m == 1: sigmas = [-38,-37]
       if m == 0.5: sigmas = [-37,-36]
       if m == 0.2: sigmas = [-36,-35,-34]      
       if shielding == 'yes':
         sigmas = sigmas[-1:]
       for sigma in sigmas:
        k += 1
        for nbins in [10]:
         for nparam in [2]:
          for det in ['Ge_migd_iZIP_bg', 'Ge_migd_HV_bg', 'Xe_migd_bg']:
           if m == 0.2 and det == 'Xe_migd_bg':
            continue
           for n_cores in [8]:
#             i += 1   
            opt = f"'-mw {m} -nlive {nlive} -target {det} -tol {tol} " \
                  f"-nparam {nparam} -bins {nbins} -notes {note} -cross_section {sigma} "\
                  f"-priors_from {p} -shielding {shielding} -verbose {verbose} -save_intermediate {save_intm} "\
                  f"-multicore_hash {unique_hash()}'"
            if save_intm == 'yes':
                t = "--walltime 96:00:00"
            else:
                t = "--walltime 96:00:00"
            if n_cores>= 116:
                mem = "--mem 7"
            else:
                mem = "--mem 3"
#             print(opt)
            n = f"--n_cores {n_cores}"
            if not np.sum(
                (np.abs(finished.mw.astype(float) - m) < 0.1) & 
                (finished.config_sigma.astype(int) == sigma) & 
                (finished.config_detector == det) &
                (finished.n_fit_parameters == nparam) & 
                (finished.config_nlive == nlive) & 
#                 (finished.config_save_intermediate == {'yes':True, 
#                                                        'no': False}[save_intm]) &
                (finished.config_earth_shielding == {'yes':True, 
                                                       'no': False}[shielding]) &
                (finished.config_notes == note)
            ):
                i += 1
                print(f'{i} m:{m} sigma:{sigma},\tdet:{det},\tnparam:{nparam},nlive:{nlive},\tintm:{save_intm}\tverne:{shielding}')
                if i>2:
                    !python write_script.py --target run_dddm_multinest.py --arguments $opt --q multicore $mem $n $t
#                 !python write_script.py --target run_dddm_multinest.py --arguments $opt --q short $mem $n 
#             assert False
print(f'done {108 - i}/108', i)

In [ ]:
hash_ = unique_hash() + "'"
!python write_script.py --target run_dddm_multinest.py --arguments '-mw 1 -nlive 1000 -target Xe_migd_bg -tol 0.9 -nparam 2 -bins 10 -notes migdal_04_03 -cross_section -37 -priors_from migdal_wide -shielding yes -verbose 1 -save_intermediate no -multicore_hash $hash_ --q multicore --mem 3 --n_cores 8 --walltime 96:00:00
# hash_ = unique_hash() + "'"
# !python write_script.py --target run_dddm_multinest.py --arguments '-mw 0.2 -nlive 1000 -target Ge_migd_HV_bg -tol 0.9 -nparam 2 -bins 10 -notes migdal_04_03 -cross_section -34 -priors_from migdal_wide -shielding yes -verbose 1 -save_intermediate no -multicore_hash $hash_ --q multicore --mem 3 --n_cores 8 --walltime 96:00:00
# hash_ = unique_hash() + "'"
# !python write_script.py --target run_dddm_multinest.py --arguments '-mw 5 -nlive 1000 -target Ge_migd_iZIP_bg -tol 0.9 -nparam 2 -bins 10 -notes migdal_04_03 -cross_section -39 -priors_from migdal_wide -shielding yes -verbose 1 -save_intermediate no -multicore_hash $hash_ --q multicore --mem 3 --n_cores 8 --walltime 96:00:00
# hash_ = unique_hash() + "'"
# !python write_script.py --target run_dddm_multinest.py --arguments '-mw 1 -nlive 1000 -target Ge_migd_HV_bg -tol 0.9 -nparam 2 -bins 10 -notes migdal_04_03 -cross_section -37 -priors_from migdal_wide -shielding yes -verbose 1 -save_intermediate no -multicore_hash $hash_ --q multicore --mem 3 --n_cores 8 --walltime 96:00:00
# hash_ = unique_hash() + "'"
# !python write_script.py --target run_dddm_multinest.py --arguments '-mw 5 -nlive 1000 -target Xe_migd_bg -tol 0.9 -nparam 2 -bins 10 -notes migdal_04_03 -cross_section -39 -priors_from migdal_wide -shielding yes -verbose 1 -save_intermediate no -multicore_hash $hash_ --q multicore --mem 3 --n_cores 8 --walltime 96:00:00        
# hash_ = unique_hash() + "'"
# !python write_script.py --target run_dddm_multinest.py --arguments '-mw 5 -nlive 1000 -target Ge_migd_HV_bg -tol 0.9 -nparam 2 -bins 10 -notes migdal_04_03 -cross_section -39 -priors_from migdal_wide -shielding yes -verbose 1 -save_intermediate no -multicore_hash $hash_ --q multicore --mem 3 --n_cores 8 --walltime 96:00:00        
# hash_ = unique_hash() + "'"
# !python write_script.py --target run_dddm_multinest.py --arguments '-mw 0.5 -nlive 1000 -target Ge_migd_iZIP_bg -tol 0.9 -nparam 2 -bins 10 -notes migdal_04_03 -cross_section -36 -priors_from migdal_wide -shielding yes -verbose 1 -save_intermediate no -multicore_hash $hash_ --q multicore --mem 3 --n_cores 8 --walltime 96:00:00        
# hash_ = unique_hash() + "'"
# !python write_script.py --target run_dddm_multinest.py --arguments '-mw 0.5 -nlive 1000 -target Ge_migd_HV_bg -tol 0.9 -nparam 2 -bins 10 -notes migdal_04_03 -cross_section -36 -priors_from migdal_wide -shielding yes -verbose 1 -save_intermediate no -multicore_hash $hash_ --q multicore --mem 3 --n_cores 8 --walltime 96:00:00        
        
        

In [ ]:
!qstat -u jorana | wc -l

In [ ]:
for xk in range(4428831, 4428837+
               ):
    !qdel $xk

In [ ]:
priors_from = ['migdal_wide']
## PROFILER
prof_dir = '/user/jorana/project/jorana/software/DD_DM_targets/submit_to_stoomboot/profiles'
tol = 0.8
# note = "TEST"
note = "migdal_27_02_mc_n"
i,k=0,0
verbose = 1
save_intm = 'yes'
# n_cores = 4
# for cores in range(4,32,4):
for n_rep in np.linspace(0,1,1):
  for p in priors_from:
   for tol in [0.9]:   
    for shielding in ["yes"]:
     for nlive in [300]:
      for m in [5,1,0.5]:
       if m == 5: sigmas = [-41,-40,-39,-38]
       if m == 1: sigmas = [-39,-38,-36, -35]
       if m == 0.5: sigmas = [-38, -37,-35, -34]
#        if m == 0.2: sigmas = [-37,-35,-33]      
       for sigma in sigmas:
        k += 1
        for nbins in [10]:
         for nparam in [2,5]:
          for det in ['Xe_migd_bg']:
           for n_cores in [12]:
            i += 1   
            opt = f"'-mw {m} -nlive {nlive} -target {det} -tol {tol} " \
                  f"-nparam {nparam} -bins {nbins} -notes {note+str(n_cores)} -cross_section {sigma} "\
                  f"-priors_from {p} -shielding {shielding} -verbose {verbose} -save_intermediate {save_intm} "\
                  f"-multicore_hash {unique_hash()}'"
            t = "--walltime 24:00:00"
            if n_cores>= 16:
                mem = "--mem 7"
            else:
                mem = "--mem 3"
            print(opt)
            n = f"--n_cores {n_cores}"
            !python write_script.py --target run_dddm_multinest.py --arguments $opt --q multicore $mem $n $t
#             assert False
i 


In [ ]:
priors_from = ['migdal_wide']
## PROFILER
prof_dir = '/user/jorana/project/jorana/software/DD_DM_targets/submit_to_stoomboot/profiles'
tol = 0.8
# note = "TEST"
note = "migdal_27_02_mc_n"
i,k=0,0
verbose = 1
save_intm = 'yes'
# n_cores = 4
# for cores in range(4,32,4):
for n_rep in np.linspace(0,1,1):
  for p in priors_from:
   for tol in [0.9]:   
    for shielding in ["yes"]:
     for nlive in [100,600]:
      for m in [5,1,0.5]:
       if m == 5: sigmas = [-41,-40,-39,-38]
       if m == 1: sigmas = [-39,-38,-36, -35]
       if m == 0.5: sigmas = [-38, -37,-35, -34]
#        if m == 0.2: sigmas = [-37,-35,-33]      
       for sigma in sigmas:
        k += 1
        for nbins in [10]:
         for nparam in [2,5]:
          for det in ['Xe_migd_bg']:
           for n_cores in [12]:
            i += 1   
            opt = f"'-mw {m} -nlive {nlive} -target {det} -tol {tol} " \
                  f"-nparam {nparam} -bins {nbins} -notes {note+str(n_cores)} -cross_section {sigma} "\
                  f"-priors_from {p} -shielding {shielding} -verbose {verbose} -save_intermediate {save_intm} "\
                  f"-multicore_hash {unique_hash()}'"
#             t = "--walltime 96:00:00"
            if n_cores>= 16:
                mem = "--mem 7"
            else:
                mem = "--mem 3"
            print(opt)
            n = f"--n_cores {n_cores}"
            !python write_script.py --target run_dddm_multinest.py --arguments $opt --q short $mem $n #$t
#             assert False
i 


In [ ]:
priors_from = ['migdal_wide']
## PROFILER
prof_dir = '/user/jorana/project/jorana/software/DD_DM_targets/submit_to_stoomboot/profiles'
tol = 0.8
# note = "TEST"
note = "migdal_27_02_mc_no_save"
i,k=0,0
verbose = 1
save_intm = 'no'
# n_cores = 4
# for cores in range(4,32,4):
for n_rep in np.linspace(0,1,1):
  for p in priors_from:
   for tol in [0.9]:   
    for shielding in ["no"]:
     for nlive in [200]:
      for m in [5,1,0.5]:
       if m == 5: sigmas = [-41,-40,-39,-38]
       if m == 1: sigmas = [-39,-38,-36, -35]
       if m == 0.5: sigmas = [-38, -37,-35, -34]
#        if m == 0.2: sigmas = [-37,-35,-33]      
       for sigma in sigmas:
        k += 1
        for nbins in [10]:
         for nparam in [2,5]:
          for det in ['Xe_migd_bg']:
           for n_cores in [12]:
            i += 1   
            opt = f"'-mw {m} -nlive {nlive} -target {det} -tol {tol} " \
                  f"-nparam {nparam} -bins {nbins} -notes {note+str(n_cores)} -cross_section {sigma} "\
                  f"-priors_from {p} -shielding {shielding} -verbose {verbose} -save_intermediate {save_intm} "\
                  f"-multicore_hash {unique_hash()}'"
            t = "--walltime 24:00:00"
            if n_cores>= 16:
                mem = "--mem 7"
            else:
                mem = "--mem 3"
            print(opt)
            n = f"--n_cores {n_cores}"
            !python write_script.py --target run_dddm_multinest.py --arguments $opt --q multicore $mem $n $t
#             assert False
i 


In [ ]:
priors_from = ['migdal_extremely_wide']

In [ ]:
## PROFILER
prof_dir = '/user/jorana/project/jorana/software/DD_DM_targets/submit_to_stoomboot/profiles'
tol = 0.8
# note = "TEST"
note = "2020_02_17_sanity_highsigma"
i,k=0,0
verbose = 1
save_intm = 'yes'
# n_cores = 4
# for cores in range(4,32,4):
for n_rep in np.linspace(0,1, 1):
  for p in priors_from:
   for tol in [0.8]:   
    for shielding in ["no"]:
     for nlive in [1000]:
      for m in [5,1,0.5]:
       if m == 5: sigmas = [-44,-43,-42,-41, -34]
       if m == 1: sigmas = [-42,-41,-40,-39,-37, -34]
       if m == 0.5: sigmas = [-37,-36, -34]
#        if m == 5: sigmas = [-42]
#        if m == 1: sigmas = [-37]
#        if m == 0.5: sigmas = [-36]
       sigmas.reverse()
       for sigma in sigmas:
        k += 1
        for nbins in [10]:
         for nparam in [2,5]:
          for det in ['Xe_migd']:
           for n_cores in [10]:
            i += 1   
            opt = f"'-mw {m} -nlive {nlive} -target {det} -tol {tol} " \
                  f"-nparam {nparam} -bins {nbins} -notes {note +str(n_cores)} -cross_section {sigma} "\
                  f"-priors_from {p} -shielding {shielding} -verbose {verbose} -save_intermediate {save_intm} "\
                  f"-multicore_hash {unique_hash()}'"
            

            if nparam ==2:
                t = "--walltime 24:00:00"
            elif nparam == 5:
                t = "--walltime 48:00:00"
            t = "--walltime 96:00:00"    
            mem = "--mem 4"
            n = f"--n_cores {n_cores}"
            print(opt)
#             !python write_script.py --target run_dddm_multinest.py --arguments $opt --q multicore $t $n $mem
i 


In [ ]:
# Submit short with shielding

## PROFILER
prof_dir = '/user/jorana/project/jorana/software/DD_DM_targets/submit_to_stoomboot/profiles'
tol = 0.8
# note = "TEST"
note = "2020_02_24_sanity_highsigma"
i,k=0,0
verbose = 1
save_intm = 'yes'
# n_cores = 4
# for cores in range(4,32,4):
for n_rep in np.linspace(0,1, 1):
  for p in priors_from:
   for tol in [0.8]:   
    for shielding in ["yes"]:
     for nlive in [300, 600]:
      for m in [5,1,0.5]:
       if m == 5: sigmas = [-44,-43,-42,-41, -34]
       if m == 1: sigmas = [-42,-41,-40,-39,-37, -34]
       if m == 0.5: sigmas = [-37,-36, -34]
#        if m == 5: sigmas = [-42]
#        if m == 1: sigmas = [-37]
#        if m == 0.5: sigmas = [-36]
       sigmas.reverse()
       for sigma in sigmas:
        k += 1
        for nbins in [10]:
         for nparam in [2,5]:
          for det in ['Xe_migd']:
           for n_cores in [8]:
            i += 1   
            opt = f"'-mw {m} -nlive {nlive} -target {det} -tol {tol} " \
                  f"-nparam {nparam} -bins {nbins} -notes {note +str(n_cores)} -cross_section {sigma} "\
                  f"-priors_from {p} -shielding {shielding} -verbose {verbose} -save_intermediate {save_intm} "\
                  f"-multicore_hash {unique_hash()}'"
            

            if nparam ==2:
                t = "--walltime 24:00:00"
            elif nparam == 5:
                t = "--walltime 48:00:00"
            t = "--walltime 4:00:00"    
            mem = "--mem 4"
            n = f"--n_cores {n_cores}"
            print(opt)
            !python write_script.py --target run_dddm_multinest.py --arguments $opt --q short $t $n $mem
i 


In [ ]:
tol = 0.8
note = "test_four_machines"
i,k=0,0
verbose = 1
save_intm = 'yes'
for n_rep in np.linspace(0,1, 1):
  for p in priors_from:
   for tol in [0.8]:   
    for shielding in ["no"]:
     for nlive in [600]:
      for m in [5,1,0.5]:
       if m == 5: sigmas = [-42]
       if m == 1: sigmas = [-37]
       if m == 0.5: sigmas = [-36]
       for sigma in sigmas:
        k += 1
        for nbins in [10]:
         for nparam in [2,5]:
          for det in ['Xe_migd']:
           for n_cores in [11]:
            i += 1   
            opt = f"'-mw {m} -nlive {nlive} -target {det} -tol {tol} " \
                  f"-nparam {nparam} -bins {nbins} -notes {note +str(n_cores)} -cross_section {sigma} "\
                  f"-priors_from {p} -shielding {shielding} -verbose {verbose} -save_intermediate {save_intm} "\
                  f"-multicore_hash {unique_hash()}'"
            t = "--walltime 96:00:00"    
            mem = "--mem 7"
            n = f"--n_cores {n_cores}"
            print(opt)
            !python write_script.py --target run_dddm_multinest.py --arguments $opt --q multicore $t $n $mem
i 


In [ ]:
tol = 0.8
note = "test_n_cores_machines"
i,k=0,0
verbose = 1
save_intm = 'yes'
for n_rep in np.linspace(0,1,2):
  for p in priors_from:
   for tol in [0.8]:   
    for shielding in ["no"]:
     for nlive in [300]:
      for m in [1]:
       if m == 5: sigmas = [-42]
       if m == 1: sigmas = [-37]
       if m == 0.5: sigmas = [-36]
       for sigma in sigmas:
        k += 1
        for nbins in [10]:
         for nparam in [2]:
          for det in ['Xe_migd']:
           for n_cores in [10,20,30,40,50]:
            i += 1   
            opt = f"'-mw {m} -nlive {nlive} -target {det} -tol {tol} " \
                  f"-nparam {nparam} -bins {nbins} -notes {note +str(n_cores)} -cross_section {sigma} "\
                  f"-priors_from {p} -shielding {shielding} -verbose {verbose} -save_intermediate {save_intm} "\
                  f"-multicore_hash {unique_hash()}'"
            t = "--walltime 96:00:00"    
            mem = "--mem 7"
            n = f"--n_cores {n_cores}"
            print(opt)
            !python write_script.py --target run_dddm_multinest.py --arguments $opt --q multicore $t $n $mem
i 


In [ ]:
## PROFILER
prof_dir = '/user/jorana/project/jorana/software/DD_DM_targets/submit_to_stoomboot/profiles'
tol = 0.8
# note = "TEST"
note = "20_02_14_long"
i,k=0,0
verbose = 10
save_intm = 'yes'
# n_cores = 4
# for cores in range(4,32,4):
for n_rep in np.linspace(0,1, 1):
  for p in priors_from:
   for tol in [0.8]:   
    for shielding in ["no"]:
     for nlive in [100,300,500]:
      for m in [5,1,0.5,0.2]:
       if m == 5: sigmas = [-44,-43,-42]
       if m == 1: sigmas = [-42,-41,-40]
       if m == 0.5: sigmas = [-39, -38, -37]
       if m == 0.2: sigmas = [-37,-35]
       for sigma in sigmas:
        k += 1
        for nbins in [10]:
         for nparam in [2,5]:
          for det in ['Xe_migd']:
#            for n_cores in [10,20]:
            i += 1   
            opt = f"'-mw {m} -nlive {nlive} -target {det} -tol {tol} " \
                  f"-nparam {nparam} -bins {nbins} -notes {note} -cross_section {sigma} "\
                  f"-priors_from {p} -shielding {shielding} -verbose {verbose} -save_intermediate {save_intm} '"

            t = "--walltime 96:00:00"
            
            print(opt)
            !python write_script.py --target run_dddm_multinest.py --arguments $opt --q long $t
#             assert False
i 


In [ ]:
## PROFILER
prof_dir = '/user/jorana/project/jorana/software/DD_DM_targets/submit_to_stoomboot/profiles'
tol = 0.8
# note = "TEST"
note = "sanity_check"
i,k=0,0
verbose = 10
save_intm = 'yes'
priors_from = ['migdal_sanity']

for n_rep in np.linspace(0,1, 1):
  for p in priors_from:
   for tol in [0.8]:   
    for shielding in ["no"]:
     for nlive in [500]:
      for m in [5,1,0.5,0.2]:
       if m == 5: sigmas = [-43,-42,-41]
       if m == 1: sigmas = [-41,-40]
       if m == 0.5: sigmas = [-37, -36]
       if m == 0.2: sigmas = [-37,-35,-34]
       for sigma in sigmas:
        k += 1
        for nbins in [10]:
         for nparam in [2,5]:
          for det in ['Xe_migd']:
           for n_cores in [20]:
            i += 1   
            opt = f"'-mw {m} -nlive {nlive} -target {det} -tol {tol} " \
                  f"-nparam {nparam} -bins {nbins} -notes {note +str(n_cores)} -cross_section {sigma} "\
                  f"-priors_from {p} -shielding {shielding} -verbose {verbose} -save_intermediate {save_intm} "\
                  f"-multicore_hash {unique_hash()}'"
            t = "--walltime 96:00:00"
#             print(opt)
         print(f'{[sigma, m]},')
#             !python write_script.py --target run_dddm_multinest.py --arguments $opt --q multicore $t --n_cores $n_cores
#             assert False
i 


In [ ]:
## PROFILER
prof_dir = '/user/jorana/project/jorana/software/DD_DM_targets/submit_to_stoomboot/profiles'
tol = 0.8
# note = "TEST"
note = "multinest_wide_prior4_short"
i,k=0,0
verbose = 10
for n_rep in np.linspace(0,1, 10):
 for p in priors_from:
  for tol in [0.8]:   
   for shielding in ["no","yes"]:
    for nlive in [200,300]:
     for m in [5,1,0.5,0.2]:
      if m == 5: sigmas = [-44,-43]
      if m == 1: sigmas = [-42,-41]
      if m == 0.5: sigmas = [-39, -38]
      if m == 0.2: sigmas = [-37]
      for sigma in sigmas:
       k += 1
       for nbins in [10]:
        for nparam in [2,5]:
         for det in ['Xe_migd']:
          i += 1   
          opt = f"'-mw {m} -nlive {nlive} -target {det} -tol {tol} " \
                f"-nparam {nparam} -bins {nbins} -notes {note +str(k)} -cross_section {sigma} "\
                f"-priors_from {p} -shielding {shielding} -verbose {verbose}'"
          t = "--walltime 96:00:00"
          print(opt)
          assert False
#           !python write_script.py --target run_dddm_multinest.py --arguments $opt --q short
#           assert False
i


In [ ]:
## PROFILER
prof_dir = '/user/jorana/project/jorana/software/DD_DM_targets/submit_to_stoomboot/profiles'
tol = 0.8
# note = "TEST"
note = "multinest_wide_prior4_generic"
i,k=0,0
verbose = 1
for n_rep in np.linspace(0,1, 2):
 for p in priors_from:
  for tol in [0.8]:   
   for shielding in ["no","yes"]:
    for nlive in [200,300]:
     for m in [5,1,0.5,0.2]:
      if m == 5: sigmas = [-44,-43]
      if m == 1: sigmas = [-42,-41]
      if m == 0.5: sigmas = [-39, -38]
      if m == 0.2: sigmas = [-37]
      for sigma in sigmas:
       k += 1
       for nbins in [10]:
        for nparam in [2,5]:
         for det in ['Xe_migd']:
          i += 1   
          opt = f"'-mw {m} -nlive {nlive} -target {det} -tol {tol} " \
                f"-nparam {nparam} -bins {nbins} -notes {note +str(k)} -cross_section {sigma} "\
                f"-priors_from {p} -shielding {shielding} -verbose {verbose}'"
          t = "--walltime 96:00:00"
          !python write_script.py --target run_dddm_multinest.py --arguments $opt --q generic
i

In [ ]:
## PROFILER
prof_dir = '/user/jorana/project/jorana/software/DD_DM_targets/submit_to_stoomboot/profiles'
tol = 0.8
# note = "TEST"
note = "multinest_wide_prior4_generic"
i,k=0,0
verbose = 10
for n_rep in np.linspace(0,1, 1):
 for p in priors_from:
  for tol in [0.8]:   
   for shielding in ["no","yes"]:
    for nlive in [100,400]:
     for m in [5,1,0.5,0.2]:
      if m == 5: sigmas = [-44,-43]
      if m == 1: sigmas = [-42,-41]
      if m == 0.5: sigmas = [-39, -38]
      if m == 0.2: sigmas = [-37]
      for sigma in sigmas:
       k += 1
       for nbins in [10]:
        for nparam in [2,5]:
         for det in ['Xe_migd']:
          i += 1   
          opt = f"'-mw {m} -nlive {nlive} -target {det} -tol {tol} " \
                f"-nparam {nparam} -bins {nbins} -notes {note +str(k)} -cross_section {sigma} "\
                f"-priors_from {p} -shielding {shielding} -verbose {verbose}'"
          t = "--walltime 96:00:00"
          !python write_script.py --target run_dddm_multinest.py --arguments $opt --q generic
i

In [ ]:
## PROFILER
prof_dir = '/user/jorana/project/jorana/software/DD_DM_targets/submit_to_stoomboot/profiles'
tol = 0.8
# note = "TEST"
note = "multinest_wide_prior4_long"
i,k=0,0
verbose = 10
for n_rep in np.linspace(0,1, 1):
 for p in priors_from:
  for tol in [0.8]:   
   for shielding in ["no","yes"]:
    for nlive in [100,400]:
     for m in [5,1,0.5,0.2]:
      if m == 5: sigmas = [-44,-43]
      if m == 1: sigmas = [-42,-41]
      if m == 0.5: sigmas = [-39, -38]
      if m == 0.2: sigmas = [-37]
      for sigma in sigmas:
       k += 1
       for nbins in [10]:
        for nparam in [2,5]:
         for det in ['Xe_migd']:
          i += 1   
          opt = f"'-mw {m} -nlive {nlive} -target {det} -tol {tol} " \
                f"-nparam {nparam} -bins {nbins} -notes {note +str(k)} -cross_section {sigma} "\
                f"-priors_from {p} -shielding {shielding} -verbose {verbose}'"
          t = "--walltime 96:00:00"
          !python write_script.py --target run_dddm_multinest.py --arguments $opt --q long $t
i

# 'Migdal' search

In [ ]:
priors_from = ['migdal_lower']

In [ ]:
## PROFILER
prof_dir = '/user/jorana/project/jorana/software/DD_DM_targets/submit_to_stoomboot/profiles'
tol = 0.8
# note = "TEST"
note = "multinest_save_intm_0203-"
i,k=0,0
verbose = 10
for p in priors_from:
  for tol in [0.1,0.8]:   
   for shielding in ["no","yes"]:
    for nlive in [200,300,500,1000]:
     for m in [5,1,0.5,0.2]:
      if m == 5: sigmas = [-44,-43]
      if m == 1: sigmas = [-42,-41]
      if m == 0.5: sigmas = [-39, -38]
      if m == 0.2: sigmas = [-37]
      for sigma in sigmas:
       k += 1
       for nbins in [10]:
        for nparam in [2,5]:
         for det in ['Xe_migd']:
          i += 1   
          opt = f"'-mw {m} -nlive {nlive} -target {det} -tol {tol} " \
                f"-nparam {nparam} -bins {nbins} -notes {note +str(k)} -cross_section {sigma} "\
                f"-priors_from {p} -shielding {shielding} -verbose {verbose}'"
          t = "--walltime 96:00:00"
          !python write_script.py --target run_dddm_multinest.py --arguments $opt --q long $t
i

In [ ]:
i


In [ ]:
## PROFILER
prof_dir = '/user/jorana/project/jorana/software/DD_DM_targets/submit_to_stoomboot/profiles'
tol = 0.8
# note = "TEST"
note = "profiler-"
i,k=0,0
verbose = 10
for p in priors_from:
  for tol in [0.8]:   
   for shielding in ["no","yes"]:
    for nlive in [50, 100]:
     for m in [1]:
#       if m == 5: sigmas = [-44,-43]
      if m == 1: sigmas = [-39]
      for sigma in sigmas:
       k += 1
       for nbins in [10]:
        for nparam in [2,5]:
         for det in ['Xe_migd']:
          i += 1   
          opt = f"'-mw {m} -nlive {nlive} -target {det} -tol {tol} " \
                f"-nparam {nparam} -bins {nbins} -notes {note +str(k)} -cross_section {sigma} "\
                f"-priors_from {p} -shielding {shielding} -verbose {verbose}'"
          !python write_script.py --target run_dddm_multinest.py --arguments $opt  --profiler True --q generic
i

In [ ]:
## PROFILER
prof_dir = '/user/jorana/project/jorana/software/DD_DM_targets/submit_to_stoomboot/profiles'
tol = 0.8
# note = "TEST"
note = "multinest_generic-"
i,k=0,0
verbose = 10
for p in priors_from:
  for tol in [0.1,0.8]:   
   for shielding in ["no","yes"]:
    for nlive in [200,300,500,1000]:
     for m in [5,1,0.5,0.2]:
      if m == 5: sigmas = [-44,-43]
      if m == 1: sigmas = [-42,-41]
      if m == 0.5: sigmas = [-39, -38]
      if m == 0.2: sigmas = [-37]
      for sigma in sigmas:
       k += 1
       for nbins in [10]:
        for nparam in [2,5]:
         for det in ['Xe_migd']:
          i += 1   
          opt = f"'-mw {m} -nlive {nlive} -target {det} -tol {tol} " \
                f"-nparam {nparam} -bins {nbins} -notes {note +str(k)} -cross_section {sigma} "\
                f"-priors_from {p} -shielding {shielding} -verbose {verbose}'"
          !python write_script.py --target run_dddm_multinest.py --arguments $opt --q generic
i

In [ ]:
## PROFILER
prof_dir = '/user/jorana/project/jorana/software/DD_DM_targets/submit_to_stoomboot/profiles'
tol = 0.8
# note = "TEST"
note = "multinest_short-"
i,k=0,0
verbose = 10
for p in priors_from:
  for tol in [0.1,0.8]:   
   for shielding in ["no","yes"]:
    for nlive in [300,500,750,1000,1500]:
     for m in [5,1,0.5,0.2]:
      if m == 5: sigmas = [-44,-43]
      if m == 1: sigmas = [-42,-41]
      if m == 0.5: sigmas = [-39, -38]
      if m == 0.2: sigmas = [-37]
      for sigma in sigmas:
       k += 1
       for nbins in [10]:
        for nparam in [2,5]:
         for det in ['Xe_migd']:
          i += 1   
          opt = f"'-mw {m} -nlive {nlive} -target {det} -tol {tol} " \
                f"-nparam {nparam} -bins {nbins} -notes {note +str(k)} -cross_section {sigma} "\
                f"-priors_from {p} -shielding {shielding} -verbose {verbose}'"
          !python write_script.py --target run_dddm_multinest.py --arguments $opt --q short
i

In [ ]:
## PROFILER
prof_dir = '/user/jorana/project/jorana/software/DD_DM_targets/submit_to_stoomboot/profiles'
tol = 0.8
# note = "TEST"
note = "short_2901-"
i,k=0,0
verbose = 10
for rep in np.linspace(0,1, 3):
    for p in priors_from:
     for sigma in [-41,-40,-39]:#-39,-38,-37,-36]:
      for tol in [0.1,0.8]:   
       for shielding in ["no","yes"]:
        for nlive in [100,200,300,400,500,750,1000]:
         k += 1
         for nbins in [10]:
          for nparam in [2,5]:
           for det in ['Xe_migd']: #'Ar_migd', 'Ge_migd',
            for m in [0.1,0.5,1]:#0.1,1,3]:
             i += 1   
             opt = f"'-mw {m} -nlive {nlive} -target {det} -tol {tol} " \
                   f"-nparam {nparam} -bins {nbins} -notes {note +str(k)} -cross_section {sigma} "\
                   f"-priors_from {p} -shielding {shielding} -verbose {verbose}'"
             !python write_script.py --target run_dddm_multinest.py --arguments $opt --q short
    #          assert False

In [ ]:
i

In [ ]:
!qstat -u jorana | wc -l


In [ ]:
## SCIENCE RESULTS

prof_dir = '/user/jorana/data/joranang/software/DD_DM_targets/submit_to_stoomboot/profiles'
tol = 0.8
note = "nestle"
note = note + "_17jan_final_results_3rd-"
i=0
verbose = 10
nlive_range = [50, 100, 200, 300, 1000, 2000, 5000]
for nlive in nlive_range:
 for p in priors_from:
  for sigma in [-38, -32]:#-39,-38,-37,-36]:
   for shielding in ['no']:
    for nparam in [2,5]:
     for nbins in [10]:
      for det in ['Xe_migd']: #'Ar_migd', 'Ge_migd',
       for m in [0.1, 1, 3]:#0.1,1,3]:
         opt = f"'-mw {m} -nlive {int(nlive)} -target {det} -tol {tol} " \
               f"-nparam {nparam} -bins {nbins} -notes {note + f'nlive_{nlive}' +str(i)} -cross_section {sigma} "\
               f"-priors_from {p} -shielding {shielding} -verbose {verbose}'"
         t = "--walltime 68:00:00"
         !python write_script.py --target run_dddm_nestle.py --arguments $opt --q generic # $t
         i+= 1
i

In [ ]:
## SCIENCE RESULTS

prof_dir = '/user/jorana/data/joranang/software/DD_DM_targets/submit_to_stoomboot/profiles'
tol = 0.8
note = "nested"
note = note + "_19jan_final_2nd_results_multinest-"
i=0
verbose = 10
nlive_range = [50, 100, 200, 300, 1000, 2000, 5000]
for nlive in nlive_range:
 for p in priors_from:
  for sigma in [-38, -32]:#-39,-38,-37,-36]:
   for shielding in ['yes','no']:
    for nparam in [5]:
     for nbins in [10]:
      for det in ['Xe_migd']: #'Ar_migd', 'Ge_migd',
       for m in [0.1, 1, 3]:#0.1,1,3]:
         opt = f"'-mw {m} -nlive {int(nlive)} -target {det} -tol {tol} " \
               f"-nparam {nparam} -bins {nbins} -notes {note + f'nlive_{nlive}' +str(i)} -cross_section {sigma} "\
               f"-priors_from {p} -shielding {shielding} -verbose {verbose}'"
#          t = "--walltime 68:00:00"
         !python write_script.py --target run_dddm_multinest.py --arguments $opt --q long
         i+= 1
#          break
i

In [ ]:
## SCIENCE RESULTS

prof_dir = '/user/jorana/data/joranang/software/DD_DM_targets/submit_to_stoomboot/profiles'
tol = 0.8
note = "nestle"
note = note + "_19jan_final_results_3rd_all-"
i=0
verbose = 10
nlive_range = [50, 100, 200, 300, 1000, 2000, 5000]
for nlive in nlive_range:
 for p in priors_from:
  for sigma in [-38, -32]:#-39,-38,-37,-36]:
   for shielding in ['yes','no']:
    for nparam in [2,5]:
     for nbins in [10]:
      for det in ['Xe_migd']: #'Ar_migd', 'Ge_migd',
       for m in [0.1, 1, 3]:#0.1,1,3]:
         opt = f"'-mw {m} -nlive {int(nlive)} -target {det} -tol {tol} " \
               f"-nparam {nparam} -bins {nbins} -notes {note + f'nlive_{nlive}' +str(i)} -cross_section {sigma} "\
               f"-priors_from {p} -shielding {shielding} -verbose {verbose}'"
         t = "--walltime 68:00:00"
#          !python write_script.py --target run_dddm_nestle.py --arguments $opt --q generic # $t
         i+= 1
i

In [ ]:
## short-assistance

prof_dir = '/user/jorana/data/joranang/software/DD_DM_targets/submit_to_stoomboot/profiles'
tol = 0.8
note = "nestle"
note = note + "_17jan_short-assistance-"
i=0
verbose = 10
nlive_range = np.linspace(200,500,50)
for nlive in nlive_range:
 for p in priors_from:
  for sigma in [-38, -32]:#-39,-38,-37,-36]:
   for shielding in ["yes"]:
    for nparam in [2,5]:
     for nbins in [10]:
      for det in ['Xe_migd']: #'Ar_migd', 'Ge_migd',
       for m in [0.1, 1, 3]:#0.1,1,3]:
         opt = f"'-mw {m} -nlive {int(nlive)} -target {det} -tol {tol} " \
               f"-nparam {nparam} -bins {nbins} -notes {note + f'nlive_{nlive}' +str(i)} -cross_section {sigma} "\
               f"-priors_from {p} -shielding {shielding} -verbose {verbose}'"
         !python write_script.py --target run_dddm_nestle.py --arguments $opt --q short
         i+= 1
i

In [ ]:
prof_dir = '/user/jorana/data/joranang/software/DD_DM_targets/submit_to_stoomboot/profiles'
tol = 0.8
note = "nestle"
note = note + "_15jan_test_restructure2dcache-"
i=0
verbose = 10
for p in priors_from:
 for sigma in [-39,-38]:#-39,-38,-37,-36]:
#  for sigma in np.linspace(-39,-35,10):   
   for shielding in ["yes"]:
    for nparam in [2,5]:
     if nparam == 5:
      nlive_range = np.linspace(100,300,151)
#       pass
     else:
      nlive_range = np.linspace(100,300,31)
     for nlive in nlive_range:#,200,300,400,500]:
      for nbins in [10]:
       for det in ['Xe_migd']: #'Ar_migd', 'Ge_migd',
        i += 1
        for m in [1]:#0.1,1,3]:
#          if nparam ==2 and nlive <50:
#             continue
         opt = f"'-mw {m} -nlive {int(nlive)} -target {det} -tol {tol} " \
               f"-nparam {nparam} -bins {nbins} -notes {note + f'nlive_{nlive}' +str(i)} -cross_section {sigma} "\
               f"-priors_from {p} -shielding {shielding} -verbose {verbose}'"
#          print(opt)
         t = "--walltime 95:00:00"
#          !echo "python write_script.py --target run_dddm_nestle.py --arguments $opt --q long"
#          !python write_script.py --target run_dddm_nestle.py --arguments $opt --q long $t
#            if i == 1:
#          !python write_script.py --target run_dddm_nestle.py --arguments $opt --q short
#          !python write_script.py --target run_dddm_nestle.py --arguments $opt --q generic
         !python write_script.py --target run_dddm_nestle.py --arguments $opt --q short
#          assert False, ""
print(3*i)

In [ ]:
prof_dir = '/user/jorana/data/joranang/software/DD_DM_targets/submit_to_stoomboot/profiles'
tol = 0.8
note = "nestle"
note = note + "short_14jan_test_restructure2dcache-"
i=0
verbose = 10
for p in priors_from:
 for sigma in [-39,-38]:#-39,-38,-37,-36]:
#  for sigma in np.linspace(-39,-35,10):   
   for shielding in ["yes"]:
    for nparam in [2,5]:
     if nparam == 5:
      nlive_range = np.linspace(100,300,251)
#       pass
     else:
      nlive_range = np.linspace(100,300,31)
     for nlive in nlive_range:#,200,300,400,500]:
      for nbins in [10]:
       for det in ['Xe_migd']: #'Ar_migd', 'Ge_migd',
        i += 1
        for m in [1]:#0.1,1,3]:
#          if nparam ==2 and nlive <50:
#             continue
         opt = f"'-mw {m} -nlive {int(nlive)} -target {det} -tol {tol} " \
               f"-nparam {nparam} -bins {nbins} -notes {note + f'nlive_{nlive}' +str(i)} -cross_section {sigma} "\
               f"-priors_from {p} -shielding {shielding} -verbose {verbose}'"
#          print(opt)
         t = "--walltime 95:00:00"
#          !echo "python write_script.py --target run_dddm_nestle.py --arguments $opt --q long"
#          !python write_script.py --target run_dddm_nestle.py --arguments $opt --q long $t
#            if i == 1:
         !python write_script.py --target run_dddm_nestle.py --arguments $opt --q short
#          !python write_script.py --target run_dddm_nestle.py --arguments $opt --q generic
#          !python write_script.py --target run_dddm_nestle.py --arguments $opt --q long $t
#         assert False, ""


In [ ]:
prof_dir = '/user/jorana/data/joranang/software/DD_DM_targets/submit_to_stoomboot/profiles'
tol = 0.8
note = "nestle"
note = note + "_tmp_migdal_probe_space_24"
i=0
verbose = 10
for p in priors_from:
 for sigma in [-39,-36]:#-39,-38,-37,-36]:
#  for sigma in np.linspace(-39,-35,10):   
   for shielding in ["yes"]:
    for nparam in [2,5]:
     if nparam == 5:
      nlive_range = np.linspace(100,300,2001)
#       pass
     else:
      nlive_range = np.linspace(100,300,1001)
     for nlive in nlive_range:#,200,300,400,500]:
      for nbins in [10]:
       for det in ['Xe_migd']: #'Ar_migd', 'Ge_migd',
        i += 1
        for m in [1]:#0.1,1,3]:
#          if nparam ==2 and nlive <50:
#             continue
         opt = f"'-mw {m} -nlive {int(nlive)} -target {det} -tol {tol} " \
               f"-nparam {nparam} -bins {nbins} -notes {note + f'nlive_{nlive}' +str(i)} -cross_section {sigma} "\
               f"-priors_from {p} -shielding {shielding} -verbose {verbose}'"
#          print(opt)
         t = "--walltime 95:00:00"
#          !echo "python write_script.py --target run_dddm_nestle.py --arguments $opt --q long"
#          !python write_script.py --target run_dddm_nestle.py --arguments $opt --q long $t
#            if i == 1:
         !python write_script.py --target run_dddm_nestle.py --arguments $opt --q short
#         assert False, ""
         print(opt)   
#          assert False

In [ ]:
prof_dir = '/user/jorana/data/joranang/software/DD_DM_targets/submit_to_stoomboot/profiles'
tol = 0.8
note = "nestle"
note = note + "_migdal_3936_long"
i=0
verbose = 10
for p in priors_from:
 for sigma in [-39,-36]:#-39,-38,-37,-36]:
#  for sigma in np.linspace(-39,-35,10):   
   for shielding in ["yes"]:
    for nparam in [2,5]:
     if nparam == 5:
      nlive_range = np.linspace(100,300,51)
#       pass
     else:
      nlive_range = np.linspace(100,300,11)
     for nlive in nlive_range:#,200,300,400,500]:
      for nbins in [10]:
       for det in ['Xe_migd']: #'Ar_migd', 'Ge_migd',
        i += 1
        for m in [1]:#0.1,1,3]:
#          if nparam ==2 and nlive <50:
#             continue
         opt = f"'-mw {m} -nlive {int(nlive)} -target {det} -tol {tol} " \
               f"-nparam {nparam} -bins {nbins} -notes {note + f'nlive_{nlive}' +str(i)} -cross_section {sigma} "\
               f"-priors_from {p} -shielding {shielding} -verbose {verbose}'"
#          print(opt)
         t = "--walltime 95:00:00"
#          !echo "python write_script.py --target run_dddm_nestle.py --arguments $opt --q long"
         !python write_script.py --target run_dddm_nestle.py --arguments $opt --q long $t
#            if i == 1:
#          !python write_script.py --target run_dddm_nestle.py --arguments $opt --q generic
#         assert False, ""
         print(opt)   
#          assert False

In [ ]:
prof_dir = '/user/jorana/data/joranang/software/DD_DM_targets/submit_to_stoomboot/profiles'
tol = 0.8
note = "nestle"
note = note + "_tmp_migdal_probe_space_24"
i=0
verbose = 10
for p in priors_from:
 for sigma in [-38]:#-39,-38,-37,-36]:
#  for sigma in np.linspace(-39,-35,10):   
   for shielding in ["yes"]:
    for nparam in [2,5]:
     if nparam == 5:
      nlive_range = np.linspace(100,300,1001)
#       pass
     else:
      nlive_range = np.linspace(100,300,501)
     for nlive in nlive_range:#,200,300,400,500]:
      for nbins in [10]:
       for det in ['Xe_migd']: #'Ar_migd', 'Ge_migd',
        i += 1
        for m in [1]:#0.1,1,3]:
#          if nparam ==2 and nlive <50:
#             continue
         opt = f"'-mw {m} -nlive {int(nlive)} -target {det} -tol {tol} " \
               f"-nparam {nparam} -bins {nbins} -notes {note + f'nlive_{nlive}' +str(i)} -cross_section {sigma} "\
               f"-priors_from {p} -shielding {shielding} -verbose {verbose}'"
#          print(opt)
         t = "--walltime 95:00:00"
#          !echo "python write_script.py --target run_dddm_nestle.py --arguments $opt --q long"
#          !python write_script.py --target run_dddm_nestle.py --arguments $opt --q long $t
#            if i == 1:
         !python write_script.py --target run_dddm_nestle.py --arguments $opt --q short
#         assert False, ""
         print(opt)   
#          assert False

In [ ]:
assert False

In [ ]:
prof_dir = '/user/jorana/data/joranang/software/DD_DM_targets/submit_to_stoomboot/profiles'
tol = 0.8
note = "nestle"
note = note + "_24th_migdal"
i=0
verbose = 10
for p in priors_from:
 for sigma in [-38, -39,-38,-37,-36]:
#  for sigma in np.linspace(-39,-35,10):   
   for shielding in ["yes"]:
    for nlive in [300,100,10]:#,200,300,400,500]:
     for nbins in [10]:
      for nparam in [2,5]:
       for det in ['Xe_migd']: #'Ar_migd', 'Ge_migd',
        i += 1
        for m in [1]:#0.1,1,3]:
#          if nparam ==2 and nlive <50:
#             continue
         note += f'nlive_{nlive}'
         opt = f"'-mw {m} -nlive {nlive} -target {det} -tol {tol} " \
               f"-nparam {nparam} -bins {nbins} -notes {note +str(i)} -cross_section {sigma} "\
               f"-priors_from {p} -shielding {shielding} -verbose {verbose}'"
#          print(opt)
         t = "--walltime 95:00:00"
#          !echo "python write_script.py --target run_dddm_nestle.py --arguments $opt --q long"
#          !python write_script.py --target run_dddm_nestle.py --arguments $opt --q long $t
#            if i == 1:
         !python write_script.py --target run_dddm_nestle.py --arguments $opt --q long --profiler False $t
#         assert False, ""
         print(opt)   

In [ ]:
note = "MCMC"
note = note + "_17th_migdal_long_full_"
i=0
verbose = 10
for p in priors_from:
  for sigma in [-39,-38,-37,-36]:
#   for sigma in np.linspace(-39,-35,10):   
   for shielding in ["yes", "no"]:
    for nsteps in [200, 400,600,800]:
     for nwalkers in [50]:
      for nbins in [10]:
       for nparam in [2,5]:
        for det in ['Xe_migd']: #, 'Ar', 'Ge'
         i += 1   
#          for m in [25, 50,250]:
         for m in [0.1,1,3]:
#           if "realistic" in p:
#             sigma = -47
#           else:
#             sigma = -45

          opt = f"'-mw {m} -nsteps {nsteps} -target {det} -nwalkers {nwalkers} " \
                f"-nparam {nparam} -bins {nbins} -notes {note +str(i)} -priors_from {p} "\
                f"-cross_section {sigma} -shielding {shielding} -verbose {verbose}'"
          print()
#           print(opt)
          t = "--walltime 95:00:00"
#           !echo "python write_script.py --target run_dddm_emcee.py --arguments $opt --q long $t"
#           !python write_script.py --target run_dddm_emcee.py --arguments $opt --q short
          !python write_script.py --target run_dddm_emcee.py --arguments $opt --q long $t
#           assert False

In [ ]:
# tmpopt = "'-mw 50 -nlive 40 -target Ar -tol 0.1 -nparam 2 -bins 10 -notes test_profiler -cross_section -45 -priors_from Pato_2010'"
# !python write_script.py --target run_dddm_nestle.py --arguments $tmpopt --q short --profiler True

# 'Normal' DD search

In [ ]:
priors_from = ['Pato_2010', 'Evans_2019_constraint', "realistic"] #'Evans_2019',

In [ ]:
# for p in priors_from:
#     note = "MCMC_19nov"
#     note = note + "_TESTER"
#     opt = f"'-mw {33} -nsteps {50} -target {'Xe'} -nwalkers {100} " \
#                 f"-nparam {2} -bins {10} -notes {note} -priors_from {p}'"
#     !python write_script.py --target run_dddm_emcee.py --arguments $opt --q short

In [ ]:
note = "MCMC"
note = note + "_update_wr"
i=0
for p in priors_from:
    for nsteps in [1000]:
     for nwalkers in [300]:
      for nbins in [10]:
       for nparam in [2,5]:
        for det in ['Xe', 'Ar', 'Ge']:
         i += 1   
         for m in [25, 50,250]:
          if "realistic" in p:
            sigma = -47
          else:
            sigma = -45

          opt = f"'-mw {m} -nsteps {nsteps} -target {det} -nwalkers {nwalkers} " \
                f"-nparam {nparam} -bins {nbins} -notes {note +str(i)} -priors_from {p} "\
                f"-cross_section {sigma}'"
          print()
          print(opt)
          t = "--walltime 95:00:00"
          !python write_script.py --target run_dddm_emcee.py --arguments $opt --q long $t

In [ ]:
# for p in priors_from:
#     note = "nestle_19nov"
#     note = note + "_TESTER"
#     tol = 0.01
#     opt = f"'-mw {33} -nlive {50} -target {'Xe'} -tol {0.5} " \
#                f"-nparam {2} -bins {10} -notes {note} -priors_from {p}'"
#     !python write_script.py --target run_dddm_nestle.py --arguments $opt --q short

In [ ]:
tol = 0.01
note = "nestle"
note = note + "_update_wr"
# for p in priors_from:
i=0
for p in priors_from:
#   for sigma in [-47]:
    for nlive in [1024*4]:
     for nbins in [10]:
      for nparam in [2,5]:
       for det in ['Ar', 'Ge', 'Xe']:
        i += 1
        for m in [25, 50, 250]:
         if "realistic" in p:
            sigma = -47
         else:
            sigma = -45
         opt = f"'-mw {m} -nlive {nlive} -target {det} -tol {tol} " \
               f"-nparam {nparam} -bins {nbins} -notes {note +str(i)} -cross_section {sigma} "\
               f"-priors_from {p}'"
         print(opt)
         t = "--walltime 95:00:00"
         !python write_script.py --target run_dddm_nestle.py --arguments $opt --q long
         print()   